# Abstract Base Classes and Operator Overloading

We often need to make a distinction between concrete classes that have a compete set of attributes and methods, and an abstract class that is missing some details.

This parallels the philosophical idea of abstraction as a way to summarize complexities.

We might say that a sailboat and an airplane have a common, abstract relationship of being vehicles, but the details of how they move are distinct.

We have two approaches to defining similar things:

**Duck Typing:**

When two class definitions have the same attributes and methods, then instances of the two classes have the same protocol and can be used interchangeably.

We often say, "when I see a bird that walks like a duck and swims like a duck and quacks like a duck, I call that bird a duck."

**Inheritance**

When two class definitions have common aspects, a subclass can share common features of a superclass.

The implementation details of the two classes should be interchangeable when we use the common features defined by superclass.

  



We can take inheritance one step further.

We can have a superclass definitions that are abstract: this means they are not directly usable by themselves, but can be used through inheritance to create concrete classes.

We have to acknowledge a terminology problem around the term base and superclass.

This is confusing because they are synonyms. There are two parallel metaphors here and we flip back and forth between them.



![Abstract](./abstract.png)


Our base class, named BaseClass, has a special class, abc.ABC, as a parent class.

This provides some special metaclass features that help make sure the concrete classes have replaced the abstractions.

The diagram shows an abstract method, `a_method`, which does not have a defined body.

A subclass must provide this.

# CREATING AN ABSTRACT BASE CLASS

Imagine we are creating a media player wtih third-party plugins.

It is advisable to create an abstract base class (ABC) in this case to document what API the third-party plugins should provide.

The greneral design is to have a common feature, like `play()`, that applies to a number of classes.

We do not want to pick some particular media format to use as a superclass; it seems somehow wrong to claim that some format is foundational and all others are derived from it.

We would prefer to define the media player as an abstraction.

Each unique kind of media file format can provide a concrete inmplementation of the abstraction.

The `àbc` module provides the tools to do this.


In [4]:
import abc

class MediaLoader(abc.ABC):
    
    @abc.abstractmethod
    def play(self) -> None:
        ...
    @property
    @abc.abstractmethod
    def ext(self) -> str:
        ...

`abc.ABC` class introduces a *metaclass* - a class used to build the concrete class definitions.

Python's default metaclass is `type`.

The default metaclass does not check for abstract methods when we try to create an instance.

The `abc.ABC` class includes an extension to the `type` metaclass to prevent us from creating instances of classes that are not fully defined.

Python uses decorators widely to make modifications to genral nature of the method or function.

In this case, it provides additional details used by the metaclass that was incldued by the `ABC` class.

Becuase we marked a method or property as abstract, any subclass of this class must implement that method or property in order to be useful, concrete implementation.

We've used the `@property` decorator on the `ext()` method, also. 

Our intent for the ext property is to provide a simple class-level variable with a string literal value. I

t's helpful to describe this as a `@property` to allow the implementation to choose between a simple variable and a method that implements the property. 

A simple variable in the concrete class will meet the expectations of the abstract class at runtime and will also help mypy to check the code for consistent use of types. 

A method could be used as an alternative to a simple attribute variable in case some more sophisticated computation is required.

One of the consequences of marking these properties is the class now has a new special attribute, `__abstractmethods__`. 

This attribute lists all of the names that need to be filled in to create a concrete class:

In [5]:
MediaLoader.__abstractmethods__

frozenset({'ext', 'play'})

See what happens if you implement a subclass? 

We'll look at an example that doesn't supply concrete implementations for the abstractions. 

We'll also look at an example that does supply the required attribute:

In [6]:
class Wav(MediaLoader):
    pass

In [7]:
x = Wav()

TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

In [8]:
class Ogg(MediaLoader):
    ext = '.ogg'
    
    def play(self):
        pass
    

In [9]:
o = Ogg()

The definition of a `Wav` subclass fails to implement either of the abstract attributes.

When we try to create an instance of the `Wav` class, an exception is raised.

Because this subclass of `MediaLoader` is still abstract, it is not possible to instantiate the class.

The class is still a potentially usefull abstract class, but you would have to subclass it and fill in the abstract placeholders before it can actually do anything.

The `Ogg` subclass supplies both attributes, so it can instantiate cleanly.

It is true, the body of the `play()` method does not do very much.

What is important is that all of the placeholders were filled, making `Ogg` a concrete subclass of the abstract ``MediaLoader` class.

 

Note that there is a subtle issue with using a class-level variable for the preferred media file extension.

Because the `ext` attribute is a varibal, it can be updated.

Using `o.ext = '.xyz'` is not expressly prohibited.

Python does not have an easy, obvious way to create read-only attributes.

We often rely on documentation to explain the consequences of changing the value of the `ext` attribute.

This has clear advantages when crating a complex application. The use of abstraction like this makes it very easy for mypy to conclude that a class does (does not) have the required methods and attributes.

This also mandates a certain amount of fussy importing to be sure that the module has access to the necessary abstract base classes for an application.

One of the advantages of duck typing is the ability to avoid complex imports and still create a useful class that can act polymorphic with peer classes.

This advantage is often outweighed by the ability of the `abc.ABC` class definition to support type checking and documentation.

the `abc.ABC` class also provides far more useful error messages when something is wrong.

One important use cas for the ABCs is the `collections` module.

This module defines the built-in generic collections using a sophisticated set of base classes and mixins.

# ABCs of Collections

A really comprehensive use of the abstract base classes in the Python standard library lives in the `collections` module.

The collections we use are extensions of the `Collection` abstract class.

`Collection` is an extension of an even more fundamental abstraction, `Container`.

Since the foundation is the `Container` class, let's inspect it in interpreter to see what method this class requires:


> import collections.abc as Container
> `Container.__abstractmethods__`

>> `frozenset({'__contains__'})`


So, the Container class has exactly one abstract method that needs to be implemented, `__contains__()`. 

You can issue `help(Container.__contains__)` to see what the function signature should look like:

`help(Container.__contains__)`
Help on function `__contains__` in module collections.abc:
`__contains__(self, x)`


We can see that `__contains__()` needs to take a single argument. 

Unfortunately, the help file doesn't tell us much about what that argument should be, but it's pretty obvious from the name of the ABC and the single method it implements that this argument is the value the user is checking to see whether the container holds.

This `__contains__()` special method implements the Python in operator. 

This method is implemented by set, list, str, tuple, and dict. 

However, we can also define a silly container that tells us whether a given value is in the set of odd integers:

In [21]:
from collections.abc import Container

class OddIntegers:
    
    def __contains__(self, x: int) -> bool:
        return x % 2 != 0

We've used the modulo test for oddity. If the remainder of x divided by two is zero, then x was even, otherwise x was odd.

Here's the interesting part: we can instantiate an `OddContainer` object and determine that, even though we did not extend `Container`, the class behaves as a `Container` object:

In [22]:
odd = OddIntegers()
isinstance(odd, Container)

True

In [23]:
issubclass(OddIntegers, Container)

True

And that is why duck typing is way more awesome than classical polymorphism. 

We can create is-a relationships without the overhead of writing the code to set up inheritance (or worse, multiple inheritance).

One cool thing about the Container ABC is that any class that implements it gets to use the `in` keyword for free. 

In fact, `in` is just syntax sugar that delegates to the `__contains__()` method. 

Any class that has a `__contains__()` method is a Container and can therefore be queried by the `in` keyword. For example:

In [24]:
odd = OddIntegers()

1 in odd

True

In [25]:
2 in odd


False

In [26]:
3 in odd

True

In [27]:
4 in odd

False

The real value here is the ability to create new kinds of collections that are completely compatible with Python's built-in generic collections. 

We could, for example, create a dictionary that uses a binary tree structure to retain keys instead of a hashed lookup. 

We'd start with the `Mapping` abstract base class definitions, but change the algorithms that support methods like `__getitem__()`, `__setitem__()`, and `__delitem__()`.

Python's duck typing works (in part) via the `isinstance()` and `issubclass()` built-in functions. 

These functions are used to determine class relationships. 

They rely on two internal methods that classes can provide: `__instancecheck__()` and `__subclasscheck__()`. 

An ABC class can provide a `__subclasshook__()` method, which is used by the `__subclasscheck__`()method to assert that a given class is a proper subclass of the abstract base class. 

The details are a bit beyond this book; consider this a signpost pointing out the path that needs to be followed when creating novel classes that need to live side by side with built-in classes.

# ABSTRACT BASE CLASSES and TYPE HINTS

The concept of an abstract base class is closely tied to the idea of a generic class.

An abstract base class is often generic with respect to some detail that is supplied by a concrete implementation.

Most of Python's generic classes – classes like list, dict, and set – can be used as type hints, and these hints can be parameterized to narrow the domain. 

There's a world of difference between `list[Any]` and `list[int]`; the value `["a", 42, 3.14]` is valid for the first type hint, but invalid for the other. 

This concept of parameterizing the generic type to make it more specific often applies to abstract classes, also.

For this to work, you will often need to incorporate `from __future__ import annotations` at the top of your module.

This is a new feature in Python 3.7 that allows you to use forward references in type hints.

This modifies the behaviour of Python to permit function and variable annotations to parameterize these standard collections.

**NOTE**

**Generic** classes and abstract classes are not the same thing. The two concepts overlap but are distinct.

Generic classes have an implicit relationship with `Any`. This often needs to be narrowed using type parameters, like `list[int]`.

The list class is concrete, and when we want to extend it, we will need to plug in a ccalss name to replace the `Any` type.

The Python interpreter does not use generic class hints in any way, they are only ckeced by static analysis tools such as mypy.

**Abstract** classes have placeholders instead of one or more methods. 

These placeholder methods require a design decision that supplies a concrete implementation.

These classes are not completely defined. When we extend it, we will need to provide a concrete method implementation.

If we do not provide the missing methods, the interpreter will raise a runtime exception when we try to create an instance of an abstract class.

Some classes can be both abstract and generic. As noted above, the type parameter helps mypy understand our intention but is not required. 

The concrete implementation is required.

Another concept that is adjacent to abstract classes is the **protocol**.

This is the essence of how duck typing works: when two classes have the same batch methods, they both adhere to a common protocol.

Any time we see classes with similar methods, there is a common protocol; this may be formalized with a type hint.

Consider objects that can be hashed.

Immutable classes implement the `__hash__()` method, including strings, integers and tuples.

Generally, mutable classes do not implement the `__hash__()` method, this includes lists, sets, and dictionaries.

If we attempt to write a type hint like `dict[list[int], list[str]]`, mypy will complain that the `list[int]` cannot be used as a key.

It cannot be a key because the given type, `list[int]`, does not implement the `Hashable` protocol.

At runtime, the attempt to create a dictionary item with a mutable key will fail for the same reason: a list does not implement the required method.



# THE collections.abc MODULE

One porminent use of abstract bassse classes is in the `collections.abc` module.

This module provides a bstract base calss definitions for Python's built in collections.

This is how `list`, `set` and `dict` (and a few other) can be built from individual component definitions.

We can use the definitions to build our own unique data structures in ways that overlap with built-in structures.

We can also use the definitions when we want to write a type hint for a specific feature of a data structure, without being overly specific about alternative implementations that might also be acceptable.

The definitions in the `collections.abc` do not include `list`, `set` and `dict`.

Instead, the module provides definitions like `MutableSequence`, `MutableSet`, and `MutableMapping`, which are abstract base classes for which `list`, `set` and `dict` classes we use are the concrete implementations.

Let's follow the various aspects of the definition of `Mapping` bact to their origins.

Python's built-in `dict` class is a concrete implementation of the `MutableMapping` abstract base class.

The abstraction comes from the idea of mapping a key to a value.

The `MutableMapping` class depends on the `Mapping` definition, an immutable, frozed dictionary, potentially optimized for lookups.

Let's follow the relationships among these abstractions.

Here's a diagram that shows the relationships among the various classes:

![MutableMapping](./MutableMapping.png)

Starting in the middle, we can see the `Mapping` definition depends on the `Collectionclass` definition. 

The definition of the Collection abstract class, in turn, depends on three other abstract base classes: `Sized`, `Iterable`, and `Container`. 

Each of these abstractions demands specific methods.

If we're going to create a lookup-only dictionary – a concrete `Mapping` implementation – we'll need to implement at least the following methods:

The `Sized` abstraction requires an implementation for the `__len__()` method. 

This lets an instance of our class respond to the `len()` function with a useful answer.


The `Iterable` abstraction requires an implementation for the `__iter__()` method.
 
This lets an object work with the for statement and the `iter()` function. 

The `Container` abstraction requires an implementation for the `__contains__() `method. 

This permits the in and not in operators to work.


The `Collection` abstraction combines Sized, Iterable, and Container without introducing additional abstract methods.

The `Mapping` abstraction, based on Collection, requires, among other things, `__getitem__()`, `__iter__()`, and` __len__()`. 

It has a default definition for `__contains__()`, based on whatever `__iter__()` method we provide. 

The Mapping definition will provide a few other methods, also.

This list of methods comes directly from the abstract relationships in the base classes. 

By building our new dictionary-like immutable class from these abstractions, we can be sure that our class will collaborate seamlessly with other Python generic classes.